In [ ]:
import pandas as pd 
import torch

df = pd.read_pickle('AnnoMI-full-with-audio-cleaned-text.pkl')

In [ ]:
# Check if cude is available then set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import soundfile as sf
import numpy as np
import os
from torchvggish import vggish, vggish_input
import h5py
import wavio
from pydub import AudioSegment
import resampy
import librosa

In [ ]:
# audio_list =  test['therapist_audio_utterance'].values[28]
# audio_list = audio_list.tolist()
# sf.write('test.wav', audio_list, 44100)

In [ ]:
# audio_list = df['client_audio_utterance'][11647] 
# audio_list = audio_list.tolist()
# audio_list += df['therapist_audio_utterance'][11648].tolist()

# sf.write('current.wav', audio_list, 44100)

In [ ]:
# # Initialise model and download weights
# embedding_model = vggish()
# embedding_model.eval()


# example = vggish_input.wavfile_to_examples('test.wav')
# embeddings = embedding_model.forward(example)

# embeddings = embeddings.detach().numpy()

In [ ]:
# Create two new columns for embeddings and initialize with None
df['client_vggish_emb'] = None
df['therapist_vggish_emb'] = None

# Initialise the VGGish model
embedding_model = vggish()
embedding_model.eval()

def resample_audio(filename, target_sr):
    y, sr = librosa.load(filename, sr=None)  # Load with the original sample rate
    y_resampled = resampy.resample(y, sr, target_sr)  # Resample to target sample rate
    sf.write(filename, y_resampled, target_sr)

# Function to compute embeddings for a given role
def compute_embeddings(role, audio_list):
    if audio_list is None:
        return None
    if len(audio_list) == 0:
        # Padding the audio
        pad_ms = 1000  # milliseconds of silence needed
        silence = AudioSegment.silent(duration=pad_ms)

        padded = silence
        padded.export('current.wav', format='wav')
        resample_audio('current.wav', 16000)

        # Compute the embeddings again
        example = vggish_input.wavfile_to_examples('current.wav')
        embeddings = embedding_model.forward(example)
        return embeddings.detach().numpy()

    audio_list = audio_list.tolist()

    # Save the audio data
    sf.write('current.wav', audio_list, 44100)
    resample_audio('current.wav', 16000)

    try:
        # Compute the embeddings
        example = vggish_input.wavfile_to_examples('current.wav')
        embeddings = embedding_model.forward(example)
        return embeddings.detach().numpy()
    except RuntimeError:
        # Padding the audio
        pad_ms = 1000  # milliseconds of silence needed
        silence = AudioSegment.silent(duration=pad_ms)
        audio = AudioSegment.from_wav('current.wav')

        padded = audio + silence  # Adding silence after the audio
        padded.export('current.wav', format='wav')
        resample_audio('current.wav', 16000)

        # Compute the embeddings again
        example = vggish_input.wavfile_to_examples('current.wav')
        embeddings = embedding_model.forward(example)
        return embeddings.detach().numpy()

In [ ]:
df_vggish = pd.DataFrame()

# Iterate through the DataFrame
for index, row in df.iterrows():
    # Determine the role (client or therapist) from the interlocutor column
    role = row['interlocutor']
    video_title = row['video_title']
    utterance = row['utterance_id']
    
    print(f'Processing {role} {utterance} {video_title}...')

    audio_list = row[f'{role}_audio_utterance']

    # Compute the embeddings for the given role
    embeddings = compute_embeddings(role, audio_list)

    # Save the embeddings in the DataFrame
    df.at[index, f'{role}_vggish_emb'] = embeddings
    df_vggish = df.copy()

In [ ]:
df.to_pickle('Annomi_VGGish.pkl')